In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn import feature_extraction, linear_model, model_selection, preprocessing
import torch

train_df = pd.read_csv("data/train.csv")
test_df = pd.read_csv("data/test.csv")

#Set device as cuda, if available
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
        
print(device)
train_df

cpu


,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1


In [3]:
test_df.head(10)

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan
5,12,NaN,NaN,We're shaking...It's an earthquake
6,21,NaN,NaN,They'd probably still show more life than Arse...
7,22,NaN,NaN,Hey! How are you?
8,27,NaN,NaN,What a nice hat?
9,29,NaN,NaN,Fuck off!


In [4]:
train_df[train_df["target"] == 0]["text"].values[1]

'I love fruits'

In [5]:
train_df[train_df["target"] == 1]["text"].values[1]

'Forest fire near La Ronge Sask. Canada'

In [6]:
import re
import string

#Cleanup the strings so we only get important words

#Remove any urls
def remove_url(text):
    url = re.compile(r'https?://\S+|www\.\S+')
    return url.sub(r'',text)

#Remove any extra html
def remove_html(text):
    html=re.compile(r'<.*?>')
    return html.sub(r'',text)

#Remove any emojis
def remove_emoji(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

#Remove puncations and hashtags
def remove_punct(text):
    table=str.maketrans('','',string.punctuation)
    return text.translate(table)

def clean_tweet(text):
    text = remove_url(text)
    text = remove_html(text)
    text = remove_emoji(text)
    text = remove_punct(text)
    return text

#Only apply this to train_df
train_df['text'] = train_df['text'].apply(lambda x : clean_tweet(x))

In [7]:
train_df.head(10)

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this earthquake Ma...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask Canada,1
2,5,NaN,NaN,All residents asked to shelter in place are be...,1
3,6,NaN,NaN,13000 people receive wildfires evacuation orde...,1
4,7,NaN,NaN,Just got sent this photo from Ruby Alaska as s...,1
5,8,NaN,NaN,RockyFire Update California Hwy 20 closed in ...,1
6,10,NaN,NaN,flood disaster Heavy rain causes flash floodin...,1
7,13,NaN,NaN,Im on top of the hill and I can see a fire in ...,1
8,14,NaN,NaN,Theres an emergency evacuation happening now i...,1
9,15,NaN,NaN,Im afraid that the tornado is coming to our area,1


In [8]:
# Split the data into train and validation sets
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

# Convert the text into vectors
vectorizer = CountVectorizer()

x_train, x_test, y_train, y_test = train_test_split(train_df['text'], train_df['target'], test_size=0.3, random_state=42)

#After splitting into train and test, tokenize
x_train_vectors = vectorizer.fit_transform(x_train)
x_test_vectors = vectorizer.transform(x_test)

In [9]:
x_train_vectors

<5329x14210 sparse matrix of type '<class 'numpy.int64'>'
	with 68468 stored elements in Compressed Sparse Row format>

In [10]:
y_train

1186    0
4071    1
5461    1
5787    1
7445    0
       ..
5226    0
5390    0
860     0
7603    1
7270    1
Name: target, Length: 5329, dtype: int64

In [11]:
from sklearn.neighbors import KNeighborsClassifier

#Just a basic kneighbors to get something working

#clf = KNeighborsClassifier(n_neighbors=3)

#clf.fit(x_train_vectors, y_train);

In [12]:
#from sklearn.metrics import accuracy_score
#y_pred = clf.predict(x_test_vectors)
#accuracy = accuracy_score(y_test, y_pred)
#print(f"Accuracy is {accuracy*100:.2f}%")

In [13]:
from torch import nn

class DisasterClassifier(nn.Module):
    def __init__(self, model):
        super(DisasterClassifier, self).__init__()
        
        self.bert = model
        #Shape of bert result is 768
        self.fc1 = nn.Linear(768, 32)
        self.relu = nn.ReLU(inplace=True)
        
        #32 nodes to binary classifier
        self.fc2 = nn.Linear(32, 1)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, input_ids, attention_mask):
        x = self.bert(input_ids=input_ids,attention_mask=attention_mask)[0][:, 0]
        x = self.fc1(x)
        x = self.relu(x)
        
        x = self.fc2(x)
        x = self.sigmoid(x)

        return x

In [37]:
import torch
from torch.optim import Adam
from tqdm.auto import tqdm, trange
import os

def train_disaster_classifier(model, train_dataloader, test_dataloader, epochs=5, _lr=.01, pretrained=''):
        
    #https://pytorch.org/docs/stable/generated/torch.nn.BCELoss.html
    criterion = nn.BCELoss()
    
    #https://pytorch.org/docs/stable/generated/torch.optim.Adam.html
    optimizer = Adam(model.parameters(), lr=_lr)
    
    model     =     model.to(device)
    criterion = criterion.to(device)
    
    if os.path.isfile(pretrained):
        model = torch.load(pretrained)
        print('Loaded model parameters from disk.')
        return model

    lowest_test_loss = float('inf')
    for epoch in range(epochs):
        total_train_accuracy = 0
        total_train_loss = 0
        
        #tell model we are gonna train
        model.train()
        
        for train_input, train_label in tqdm(train_dataloader):
            
            #Put label on cuda or cpu 
            train_label = train_label.to(device)
            
            #https://huggingface.co/docs/transformers/glossary#attention-mask
            #Get attention mask and input ids from BERT model
            #Basically pads length of strings so they are equal size
            attention_mask = train_input['attention_mask'].to(device)
            
            #https://huggingface.co/docs/transformers/glossary#token-type-ids
            #token indices, numerical representations of tokens building the sequences that will be used as input by the model.
            input_ids      = train_input['input_ids'].squeeze(1).to(device)

            output = model(input_ids, attention_mask)
            loss = criterion(output, train_label.float().unsqueeze(1))
            print(output)
            total_train_loss += loss.item()

            accuracy = ((output >= 0.5).int() == train_label.unsqueeze(1)).sum().item()
            total_train_accuracy += accuracy

            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

        with torch.no_grad():
            total_test_accuracy = 0
            total_test_loss = 0
            
            model.eval()
            
            for test_input, test_label in tqdm(test_dataloader):
                test_label = test_label.to(device)
                
                attention_mask = test_input['attention_mask'].to(device)
                input_ids = test_input['input_ids'].squeeze(1).to(device)

                output = model(input_ids, attention_mask)
                
                loss = criterion(output, test_label.float().unsqueeze(1))

                total_test_loss += loss.item()

                accuracy = ((output >= 0.5).int() == test_label.unsqueeze(1)).sum().item()
                total_test_accuracy += accuracy
            
            print(f'Epochs: {epoch}\nTraining -> loss: {(total_train_loss / len(train_dataloader)): .3f} '
                  f'accuracy: {(total_train_accuracy / (len(train_dataloader.dataset)))*100: .2f}%')
            
            print(f'Testing -> loss: {total_test_loss / len(test_dataloader): .3f} '
                  f'accuracy: {(total_test_accuracy / len(test_dataloader.dataset))*100: .2f}%')
            
            if lowest_test_loss > total_test_loss:
                lowest_test_loss = total_test_loss
                torch.save(model, f"best_accuracy_model2.pt")
                print("Saved model")

In [38]:
from torch.utils.data import Dataset

TWEET_LEN = 280

class DisasterDataset(Dataset):
    def __init__(self, text_df, label_df, tokenizer):
        texts = text_df.values.tolist()

        texts = [clean_tweet(text) for text in texts]
               
        #https://huggingface.co/docs/transformers/main/main_classes/tokenizer
        #  padding = 'max_length': Pad to a maximum length specified with the argument max_length 
        #  max_length = 280: Max length to use by one of the parameters. 280 since max length of tweets
        #  truncation = True: Truncate to max lenght set by max_length
        #  return_tensors="pt": Return a PyTorch tesnor
        self.texts = [tokenizer(text, max_length=TWEET_LEN, padding='max_length', truncation=True, return_tensors="pt")
                      for text in texts]
        self.labels = label_df.values.tolist()
        
    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        return text, label

In [39]:
#https://huggingface.co/docs/transformers/main_classes/tokenizer

from transformers import AutoTokenizer, AutoModel
from torch.utils.data import DataLoader
import warnings
warnings.filterwarnings('ignore')

torch.manual_seed(0)
np.random.seed(0)
     
#https://huggingface.co/docs/transformers/model_doc/roberta
tokenizer = AutoTokenizer.from_pretrained("roberta-base")
model     = AutoModel.from_pretrained("roberta-base")

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [44]:
train_dataloader = DataLoader(DisasterDataset(x_train, y_train, tokenizer), batch_size=8, shuffle=True, num_workers=0)
test_dataloader = DataLoader(DisasterDataset(x_test, y_test, tokenizer), batch_size=8, num_workers=0)

model = DisasterClassifier(model)

model = train_disaster_classifier(model, train_dataloader, test_dataloader, epochs=1, _lr=1e-3, pretrained='best_accuracy_model.pt')

In [45]:
sample_submission = pd.read_csv("data/sample_submission.csv")

In [46]:
#Only apply this to train_df
test_df['text'] = test_df['text'].apply(lambda x : clean_tweet(x))

tokenizer = AutoTokenizer.from_pretrained("roberta-base")

ktest_dataloader = DataLoader(DisasterDataset(test_df['text'], train_df['target'], tokenizer), batch_size=8, num_workers=0)

In [52]:
tweet_pred = []
with torch.no_grad():
    model.eval()
    for test_input, test_label in tqdm(ktest_dataloader):
        attention_mask = test_input['attention_mask'].to(device)
        input_ids = test_input['input_ids'].squeeze(1).to(device)
        output = model(input_ids, attention_mask)
        
        tweet_pred.append((output[:,0] >= 0.5).numpy().astype(int))

  0%|          | 0/408 [00:00<?, ?it/s]

In [53]:
print(np.concatenate(tweet_pred))

[1 1 1 ... 1 1 1]


In [54]:
sample_submission["target"] = np.concatenate(tweet_pred)

In [55]:
sample_submission.head()

,id,target
0,0,1
1,2,1
2,3,1
3,9,1
4,11,1


In [56]:
sample_submission.to_csv("submission.csv", index=False)